# How to create more complex workflows

## Concatenating several scripts to one workflow and more :)

In the previous notebook, we have seen how we can run arbitrary executables through `aiida-shell` without requiring any
code-specific infrastructure (typically contained in a dedicated AiiDA plugin). In addition, we have seen how we can
feed the output of one task to the input of another task, linking the two and effectively creating a workflow.

Building on this concept, the `aiida-workgraph` provides the capability to create workflows in the same manner as one would
build up an actual graph. That is, by adding nodes and edges to it. It further extends the possible building blocks from
external scripts to existing AiiDA buliding blocks (`CalcFunction`s, `CalcJob`s, `WorkChain`s, etc.), as well as custom
Python code.

We'll cover lots of material in this notebook, so strap yourself in and buckle up! :rocket:

To run the following Python cells, we need to make sure that we select the correct kernel `Python3.10 (AIIDA)`. If it is
not already selected, do so as follows:

<img src="../../data/figs/change_notebook_kernel.png" width="500" style="height:auto; display:block; margin-left:auto; margin-right:auto;">

We then load the AiiDA jupyter notebook extension, check the profile status, import the libraries all that we need. So nothing new
here, really...

In [ ]:
%load_ext aiida
%aiida

In [ ]:
%verdi status

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import time

from aiida import orm
from aiida_shell.parsers import ShellParser
from aiida.tools.visualization import Graph

from aiida_workgraph import WorkGraph, task
from aiida_workgraph.utils import generate_node_graph

In [ ]:
diag_code = orm.load_code('diagonalization@localhost')  # The computer label can also be omitted here
query_code = orm.load_code('remote_query@localhost')  # The computer label can also be omitted here
db_path = str(Path('../../data/euro-scipy-2024/diag-wf/remote/matrices.db').resolve())

In [ ]:
def provenance_graph(aiida_node):
    graph = Graph()
    graph.recurse_ancestors(aiida_node, annotate_links="both")
    graph.recurse_descendants(aiida_node, annotate_links="both")
    display(graph.graphviz)

## WorkGraph vs. provenance graph

As evident from the import statement:

```python
from aiida_workgraph import WorkGraph, task
```

the first entity we'll be using is, of course, the `WorkGraph`. In addition, we import the `task`, which actually
presents the `WorkGraph` equivalent of a *node* in the graphs we'll be building up.

In line with common graph nomenclature, we'd have loved to use the **Node** keyword for that, but remember, the `Node`
class is already defined in `aiida-core`. To avoid confusion, it is good to mention here, that we will now be talking
about two different kinds of graphs:
- **The provenance graph**: AiiDA's way of storing the **Data** and **Processes** inside the SQL database as **Node**s
  and **Link**
- **The WorkGraph**: The workflow we are building up using the `aiida-workgraph` library

As such, we can build up our workflow as a **WorkGraph**, run it, and AiiDA will store all data in its database, allowing
us to explore the resulting **provenance graph** of our workflow.

Let's maybe best start with some simple examples, this will make things clear. We'll close the cycle to the previous
notebook in a bit.

In [ ]:
def sleep_and_print(sleep_time, print_statement):
    time.sleep(sleep_time)
    print(print_statement)

wg = WorkGraph('First WG')

wg.add_task(sleep_and_print)
wg

Congratulations, you just created your first `WorkGraph`! Let's unpack the code: We first created a very simple Python
function, we then instantiated the `WorkGraph`, and added our function as a task (remember, think of *graph nodes*).

`aiida-workgraph` comes with a visualization tool in which we can see the setup of our workflow. Note that we didn't
actually run it at this point, yet. Let's add a second task:

In [ ]:
wg.add_task(sleep_and_print)
wg

We can see that we now have two disconnected tasks in our workgraph. To define dependencies between those, we can either
link inputs and outpus, just as we did before with `aiida-shell`, or explicitly enforce that the second task has to wait
on the first one. For now, let's actually focus on the second case (the first one will require us to introduce a few
more concepts):

In [ ]:
wg.tasks.sleep_and_print2.waiting_on.add('sleep_and_print1')
wg

In the cell above, we accessed the second task through our `WorkGraph` instance, `wg`. However, the `add_task` function
actually returns the task, so we can also write:

In [ ]:
wg = WorkGraph('First WG')

task1 = wg.add_task(sleep_and_print)
task2 = wg.add_task(sleep_and_print)

task2.waiting_on.add('sleep_and_print1')
wg

which achieves the same.

## Running Python code with WorkGraph and AiiDA provenance


If we want to actually run our workflow, we should specify some inputs to our tasks. We can do that, as well as name our
tasks like so:

In [ ]:
wg = WorkGraph("Run WG")

task1 = wg.add_task(
    sleep_and_print, name="lets_start", sleep_time=1, print_statement="Let's start"
)

task2 = wg.add_task(
    sleep_and_print, name="lets_continue", sleep_time=1, print_statement="Let's continue"
)

task2.waiting_on.add('lets_start')

display(wg)
wg.run()

Seems like everything worked out smoothly. Now, let's show the provenance graph of our workflow:

In [ ]:
provenance_graph(aiida_node=wg)

But where are our tasks? :anguished:

It is important to note here that AiiDA does not store the plain Python function we used to define our tasks in its
database. Remember, the AiiDA classes derived from `Node` implement this functionality, so AiiDA doesn't know how to
store the data in the database.  Thankfully, we can easily resolve this issue by adding the `@task.calcfunction`
decorator to our `sleep_and_print` function. We now assume that we are dealing with AiiDA `ORM` data types inside the
task, so we access their actual `value`s inside the function 

In [ ]:
@task.calcfunction
def sleep_and_print(sleep_time, print_statement):
    time.sleep(sleep_time.value)
    print(print_statement.value)

wg = WorkGraph("Provenance restored")

task1 = wg.add_task(
    sleep_and_print, name="lets_start", sleep_time=1, print_statement="Let's start"
)

task2 = wg.add_task(
    sleep_and_print, name="lets_continue", sleep_time=1, print_statement="Let's continue"
)

task2.waiting_on.add('lets_start')

display(wg)
wg.run()

In [ ]:
provenance_graph(aiida_node=wg)

## On creating, returning, and linking data

Now, if we would like to specify data dependencies, we should define a `task.calcfunction` that actually
returns some output so that we can then link it as an input to another task (before, we were only printing).

The function in the next cell achieves just that. Here, we have manually specified our `outputs` in the decorator, and
we return a clone of the `print_statement`, as returning the actual data node would create a cycle in the graph, which
is forbidden:

In [67]:
@task.calcfunction(
    outputs=[
        {'name': 'return_statement'}
    ]
)
def sleep_and_return(sleep_time, print_statement):
    time.sleep(sleep_time.value)
    # Returning the input directly would create a cycle in the graph
    return {'return_statement': print_statement.clone()}

wg = WorkGraph("Linked data")

return_task = wg.add_task(
    sleep_and_return, name="return_task", sleep_time=1, print_statement="I will print the previous return."
)

task5 = wg.add_task(
    sleep_and_print, name="actual_print_task", sleep_time=1, print_statement=return_task.outputs['return_statement']
)

display(wg)
wg.run()

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'Linked …

update task state:  return_task
update task state:  actual_print_task
Continue workgraph.
Report: [84916|WorkGraphEngine|continue_workgraph]: Continue workgraph.


REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[84916|WorkGraphEngine|continue_workgraph]: Continue workgraph.


Report: [84916|WorkGraphEngine|continue_workgraph]: tasks ready to run: return_task


REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[84916|WorkGraphEngine|continue_workgraph]: tasks ready to run: return_task


...

REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[84916|WorkGraphEngine|continue_workgraph]: tasks ready to run: 


task:  return_task FINISHED
task:  actual_print_task FINISHED
is workgraph finished:  True
workgraph outputs:  []
Report: [84916|WorkGraphEngine|finalize]: Finalize


REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[84916|WorkGraphEngine|finalize]: Finalize


Finalize workgraph Linked data



{'new_data': {},
 'execution_count': <Int: uuid: 6e9f08e5-28d8-4ad6-a259-f79518a2f3df (pk: 84923) value: 0>}

With this, we can now define (almost) arbitrarily complex workflows, as shown below. Feel free to play around with this!

In [69]:
wg = WorkGraph("Arbitrary WorkGraph")

task1 = wg.add_task(
    sleep_and_print, name="lets_start", sleep_time=1, print_statement="Let's start"
)

task2 = wg.add_task(
    sleep_and_print,
    name="lets_continue",
    sleep_time=1,
    print_statement="Let's continue",
)

task2.waiting_on.add("lets_start")

task3 = wg.add_task(
    sleep_and_print,
    name="wait_both",
    sleep_time=1,
    print_statement="I need to wait for both",
)

task3.waiting_on.add("lets_start")
task3.waiting_on.add("lets_continue")

disconnected_task = wg.add_task(
    sleep_and_print,
    name="disconnected_task",
    sleep_time=5,
    print_statement="I have no dependencies, but I am one, and I take my time.",
)

task4 = wg.add_task(
    sleep_and_return,
    name="intermediate_step",
    sleep_time=1,
    print_statement="I will print the previous return.",
)

task4.waiting_on.add("disconnected_task")
task4.waiting_on.add("wait_both")

task5 = wg.add_task(
    sleep_and_print,
    name="final_step",
    sleep_time=1,
    print_statement=task4.outputs["return_statement"],
)

display(wg)
wg.run()

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'Arbitra…

update task state:  lets_start
update task state:  lets_continue
update task state:  wait_both
update task state:  disconnected_task
update task state:  intermediate_step
update task state:  final_step
Continue workgraph.
Report: [84924|WorkGraphEngine|continue_workgraph]: Continue workgraph.


REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[84924|WorkGraphEngine|continue_workgraph]: Continue workgraph.


Report: [84924|WorkGraphEngine|continue_workgraph]: tasks ready to run: lets_start,disconnected_task


REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[84924|WorkGraphEngine|continue_workgraph]: tasks ready to run: lets_start,disconnected_task


...

REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[84924|WorkGraphEngine|continue_workgraph]: tasks ready to run: 


task:  lets_start FINISHED
task:  lets_continue FINISHED
task:  wait_both FINISHED
task:  disconnected_task FINISHED
task:  intermediate_step FINISHED
task:  final_step FINISHED
is workgraph finished:  True
workgraph outputs:  []
Report: [84924|WorkGraphEngine|finalize]: Finalize


REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[84924|WorkGraphEngine|finalize]: Finalize


Finalize workgraph Arbitrary WorkGraph



{'new_data': {},
 'execution_count': <Int: uuid: acb1d6ab-7090-45e7-9afa-3e45202d1eaf (pk: 84943) value: 0>}

## Closing the circle: Back to the `aiida-shell` example

Now that we have seen how we can construct simple workflows and define task dependencies with the `WorkGraph`, let's use
it to implement the workflow from the previous notebook. The code snippets in the following cells are rather lengthy,
however, the way we execute the external executable is the same as before, just that we now add a `ShellJob` `task` to
the `WorkGraph` (passing the same arguments as before):

In [ ]:
wg = WorkGraph("query_and_diag")

matrix_pk = 5
query_output_filename = f"matrix-{matrix_pk}.npy"

query_task = wg.add_task(
    "ShellJob",
    name="query_task",
    command=query_code,
    arguments=["{db_path}", "{matrix_pk}"],
    nodes={"db_path": db_path, "matrix_pk": orm.Int(matrix_pk)},
    outputs=[query_output_filename],
)

# The file name automatically gets converted into an AiiDA link label by `aiida-shell`
# Link labels can only have alphanumericy characters and underscores, so we apply the same cleaning to the filename
# To be able to reference it later on
query_task_link_label = ShellParser.format_link_label(query_output_filename)

### Attaching a parser

Now that we have run the query task as before, the next step is the diagonalization. However, we might not only want to
write the eigenvalues to an output file, but also parse them, e.g. so that the resulting array is stored
**explicitly** in AiiDA's database (rather than just a reference to the file), and so that we can further operate on it
directly in our Python code. To achieve that, we deine a parser function:

In [ ]:
def parse_array(self, dirpath: Path) -> dict[str, orm.Data]:
    arr = np.loadtxt(dirpath / self.node.inputs.outputs[0])
    data = orm.ArrayData(arr)
    return {"eigvals": data}

Which we can now pass to our diagonalization task via:

In [ ]:
diag_output_filename = f"matrix-{matrix_pk}-eigvals.txt"

diag_task = wg.add_task(
    "ShellJob",
    name="diag_task",
    command=diag_code,
    arguments=["{matrix_file}"],
    nodes={"matrix_file": query_task.outputs[query_task_link_label]},
    outputs=[diag_output_filename],
    # Attach parser here
    parser=parse_array,
    parser_outputs=[{"name": "eigvals"}],
)
diag_task_link_label = ShellParser.format_link_label(diag_output_filename)

display(wg)
wg.run()

This now allows us to retrieve the eigenvalue outputs directly from the associated AiiDA `Node` attached to the
`WorkGraph` `Task`:

In [ ]:
diag_task.node.outputs['aiida_shell_5_eigvals_txt']
diag_task.node.outputs.eigvals.get_array()

## Extending WorkGraph with arbitrary python code

In [ ]:
wg = WorkGraph("compute_eigvals_wg")
matrix_pk = 5
query_output_filename = f"matrix-{matrix_pk}.npy"
query_task = wg.add_task(
    "ShellJob",
    name="query_task",
    command=query_task,
    arguments=["{db_path}", "{matrix_pk}"],
    nodes={"db_path": db_path, "matrix_pk": orm.Int(matrix_pk)},
    outputs=[query_output_filename],
)

query_task_link_label = ShellParser.format_link_label(query_output_filename)
diag_output_filename = f"matrix-{matrix_pk}-eigvals.txt"


def parse_array(self, dirpath: Path) -> dict[str, orm.Data]:
    global diag_output_filename
    arr = np.loadtxt(dirpath / diag_output_filename)
    return {"eigvals": orm.ArrayData(arr)}


diag_task = wg.add_task(
    "ShellJob",
    name="diag_task",
    command=diag_task,
    arguments=["{matrix_file}"],
    parser=parse_array,
    nodes={"matrix_file": query_task.outputs[query_task_link_label]},
    outputs=[diag_output_filename],
    parser_outputs=[{"name": "eigvals"}],
)

diag_task_link_label = ShellParser.format_link_label(diag_output_filename)


# Why do you have to wrap your function? So aiida understands your function
# Try comment out the code and look at the provenance graph
@task.calcfunction
def compute_mean(eigenvalues: orm.ArrayData) -> dict[str, orm.Data]:
    return orm.Float(np.mean(eigenvalues.get_array()))


# plot_task = wg.add_task(
#     plot, name="plot_task", eigenvalues=diag_task.outputs["eigvals"]
# )
mean_task = wg.add_task(
    compute_mean, name="mean_task", eigenvalues=diag_task.outputs["eigvals"]
)

wg.run()

#### We can see that the `compute_mean` result (the orm.Float) is not present in the provenance graph when we remove the calcfunction decorator because it is not stored in the database.

In [ ]:
from aiida_workgraph.utils import generate_node_graph
generate_node_graph(wg.pk)

#### We can display the image in a similar way by 

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Image, display

print(wg.tasks["plot_task"].outputs["result"].value)  # SinglefileData
with wg.tasks["plot_task"].outputs["result"].value.as_path() as filepath:
    display(Image(filename=(filepath)))

In [ ]:
def array_parser(self, dirpath: Path) -> dict[str, orm.Data]:
    arr = np.loadtxt(dirpath / self.node.inputs.outputs[0])  # this is small aiida detail
    data = orm.ArrayData(arr)
    data.attributes["length"] = len(arr)
    return {"eigvals": data}


@task.graph_builder(
    outputs=[
        {"name": "eigvals", "from": "diag_task.eigvals"},
        {"name": "mean_eigval", "from": "compute_mean.result"},
    ]
)
def query_and_diag(matrix_pk: orm.Int):
    wg = WorkGraph()
    query_output_filename = f"matrix-{matrix_pk.value}.npy"

    query_code = orm.load_code(f"query@localhost")
    query_task = wg.add_task(
        "ShellJob",
        name="query_task",
        command=query_code,
        arguments=["{db_path}", "{matrix_pk}"],
        nodes={
            "db_path": "/Users/alexgo/code/fair-workflows-workshop/data/euro-scipy-2024/diag-wf/remote/matrices.db",
            "matrix_pk": matrix_pk,
        },
        outputs=[query_output_filename],
    )
    query_task_link_label = ShellParser.format_link_label(query_output_filename)
    diag_output_filename = f"matrix-{matrix_pk.value}-eigvals.txt"

    diag_code = orm.load_code(f"diag@localhost")
    wg.add_task(
        "ShellJob",
        name="diag_task",
        command=diag_code,
        arguments=["{matrix_file}"],
        parser=array_parser,
        nodes={"matrix_file": query_task.outputs[query_task_link_label]},
        outputs=[diag_output_filename],
        parser_outputs=[{"name": "eigvals"}],
    )

    @task.calcfunction
    def compute_mean(eigenvalues: orm.ArrayData) -> dict[str, orm.Data]:
        node = orm.Float(np.mean(eigenvalues.get_array()))
        node.attributes["length"] = len(eigenvalues)
        return node

    # TODO add compute_mean

    return wg


wg = WorkGraph()
wg.add_task(query_and_diag)
wg

In [ ]:
wg = WorkGraph("processing_data")
for i in range(1, 5):
    query_and_diag_task = wg.add_task(
        query_and_diag, name=f"query_and_diag_pk{i}", matrix_pk=orm.Int(i)
    )
display(wg)
wg.run()

### We want to collect all the results and plot them

In [ ]:
# TODO you don't have to compute anymore the mean value because it is exposed by the graph_builder
@task.calcfunction
def assemble_plot(**collected_eigvals) -> dict[str, orm.Data]:
    # return orm.List([arr.get_array() for arr in x.values()])
    fig, ax = plt.subplots(figsize=(8, 6))
    label: str
    eigval_data: orm.ArrayData
    mean_eigenvalues = []
    for _, eigval_data in collected_eigvals.items():
        mean_eigenvalues.append(np.mean(eigval_data.get_array()))
    ax.hist(mean_eigenvalues, bins=10, color="c", edgecolor="black")
    ax.set_title("Histogram of Eigenvalues")
    ax.set_xlabel("Eigenvalue")
    filename = "plot.jpg"
    plt.legend()
    plt.savefig(filename)
    plt.close(fig)
    return orm.SinglefileData(Path(filename).absolute())

In [ ]:
wg = WorkGraph("processing_data")
assemble_plot_task = wg.add_task(assemble_plot, name="assemble_plot_task")
# we have to increase the link limit because by default workgraph only supports one link per input socket
assemble_plot_task.inputs["collected_eigvals"].link_limit = 50
for i in range(1, 10):
    query_and_diag_task = wg.add_task(
        query_and_diag, name=f"query_and_diag_pk{i}", matrix_pk=orm.Int(i)
    )
    wg.add_link(
        query_and_diag_task.outputs["eigvals"],
        assemble_plot_task.inputs["collected_eigvals"],
    )
display(wg)
wg.run()

## QueryBuilder

### We can query now from our results

In [ ]:
# TODO QueryBuilder expand plot something
qb = orm.QueryBuilder()
qb.append(
    orm.ArrayData,
    project=['attributes.array|default']
)
qb.all(flat=True)


### We can also introduce filters in our queriesS

In [ ]:
# TODO QueryBuilder expand plot something
qb = orm.QueryBuilder()
qb.append(
    orm.ArrayData,
    filters={
        'attributes.length': {'==': 50}
    },
    project=['attributes.array|default']
)
qb.all(flat=True)

## How can we create if conditions workflows?

In [ ]:
@task.calcfunction
def compute_mean(eigvals: orm.ArrayData) -> orm.Float:
    return orm.Float(np.mean(eigvals.get_array()))


@task.calcfunction
def eigvals_less(mean_eigval: orm.Float) -> bool:
    return mean_eigval < 14.5


@task.calcfunction
def heureka(eigvals, pk):
    try:
        path = Path("storage").absolute()
        path.mkdir(exist_ok=True)
        result_path = path / f"eigvals-pk{pk}.npy"
        np.save(result_path, eigvals.get_array())
        success = orm.Int(0)
        success.attributes["path"] = str(result_path)
        success.attributes["error"] = ""
    except Exception as err:
        success = orm.Int(1)
        success.attributes["path"] = ""
        success.attributes["error"] = str(err)
    return success

In [ ]:
wg = WorkGraph("someother")

martix_pk = 5

query_and_diag_task = wg.add_task(
    query_and_diag, name=f"query_and_diag_pk{matrix_pk}", matrix_pk=orm.Int(matrix_pk)
)
compute_mean_task = wg.add_task(
    compute_mean,
    name=f"compute_mean_pk{matrix_pk}",
    eigvals=query_and_diag_task.outputs["eigvals"],
)
eigvals_less_task = wg.add_task(
    eigvals_less,
    name=f"eigvals_less_task_pk{matrix_pk}",
    mean_eigval=compute_mean_task.outputs["result"],
)
if_less = wg.add_task(
    "If", name=f"if_less_pk{matrix_pk}", conditions=eigvals_less_task.outputs["result"]
)  # there as specific conditions socket
heureka_task = wg.add_task(
    heureka,
    name=f"heureka_task_pk{matrix_pk}",
    eigvals=query_and_diag_task.outputs["eigvals"],
    pk=orm.Int(i),
)
if_less.children.add(f"heureka_task_pk{matrix_pk}")

display(wg)
wg.run()